In [1]:
%%capture
import torch
import numpy as np
from torch.utils.data import DataLoader
from catalyst.dl import SupervisedRunner
from catalyst.dl.utils import set_global_seed, prepare_cudnn
from catalyst.dl.callbacks import AccuracyCallback
set_global_seed(0)
prepare_cudnn(deterministic=True)
from pathlib import Path
from tqdm import tqdm

In [2]:
class BVPDataset(torch.utils.data.Dataset):

    def __init__(self, is_train: bool = False, with_x: bool = False):
        path = Path('dataset').absolute()
        filepaths = list(path.rglob('*.npy'))
        filepaths = sorted(filepaths, key=lambda x: int(x.stem))
        self.filepaths = filepaths[int(is_train)::2]
        self.with_x = with_x

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, index):
        filepath = self.filepaths[index]
        sample = np.load(filepath.as_posix())
        sample = torch.FloatTensor(sample)
        inputs = sample[1]
        targets = sample[2]
        if self.with_x:
            xxx = sample[0]
            return xxx, inputs, targets
        else:
            return inputs, targets

In [11]:
class BVPNet(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(100, 1024)
        self.fc2 = torch.nn.Linear(1024, 100)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.tanh(x)
        x = self.fc2(x)
        return x

    # def __len__(self):
    #     return sum(p.numel() for p in self.parameters() if p.requires_grad)

    # def init_weights(self, filepath):
    #     return self.load_state_dict(torch.load(filepath))

    # def save_weights(self, filepath):
    #     return torch.save(self.state_dict(), filepath)

In [16]:
train_dataset = BVPDataset(True)
valid_dataset = BVPDataset(False)
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
loaders = {'train': train_loader, 'valid': valid_loader}

torch.Size([10, 100])


In [17]:
model = BVPNet()
# print('Parameters:', len(model))
print(model)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=120, gamma=0.8)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
runner = SupervisedRunner(device=device)
callbacks = [
    AccuracyCallback(),
]

Parameters: 205924
BVPNet(
  (fc1): Linear(in_features=100, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=100, bias=True)
)


In [18]:
logdir = 'logs'
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    # scheduler=scheduler,
    loaders=loaders,
    callbacks=callbacks,
    logdir=logdir,
    num_epochs=10,
    verbose=True,
)

[2020-05-30 17:04:49,402] 
1/10 * Epoch 1 (_base): lr=0.0100 | momentum=0.9000
1/10 * Epoch 1 (train): accuracy01=0.1168 | loss=44.6703
1/10 * Epoch 1 (valid): accuracy01=0.2107 | loss=17.6892
[2020-05-30 17:04:54,454] 
2/10 * Epoch 2 (_base): lr=0.0100 | momentum=0.9000
2/10 * Epoch 2 (train): accuracy01=0.2559 | loss=15.8468
2/10 * Epoch 2 (valid): accuracy01=0.2577 | loss=9.0787


In [ ]:
def run_epoch(model, criterion, optimizer, loaders, device, stage):
    is_train = stage == 'train'
    if is_train: model.train()
    else: model.eval()
    losses = []
    with torch.set_grad_enabled(is_train):
        for xxx, inputs, targets in tqdm(loaders[stage]):
            inputs = inputs.to(device)
            targets = targets.to(device)
            preds = model(inputs)
            loss = criterion(preds, targets)
            losses.append(loss.detach().cpu().numpy())
            if is_train:
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
    # if is_train:
    #     scheduler.step()
    mean_loss = round(np.mean(losses), 8)
    return {f'{stage}_loss': mean_loss}

In [ ]:
model = model.to(device)
log = ''
best_loss = 1e5
for epoch in range(666):
    out = {'epoch': epoch + 1}
    train_loss = run_epoch(model, criterion, optimizer, loaders, device, 'train')
    out.update(train_loss)
    valid_loss = run_epoch(model, criterion, optimizer, loaders, device, 'valid')
    out.update(valid_loss)
    out.update({'lr': scheduler.get_last_lr()})
    t = f'{out}\n'
    # print('\n', t)
    log += t
    if out['valid_loss'] < best_loss:
        model.save_weights('best_model.pth')
    with open('log.txt', 'w') as f:
        f.write(log)

In [ ]:
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

def plot(xxx, inputs, targets, preds):
    fig = plt.figure(figsize=(4, 3))
    ax = fig.gca()
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_xlim([xxx.min(), xxx.max()])
    ax.grid()
    ax.plot(xxx, inputs, label='Y(x)')
    ax.plot(xxx, targets, label='F(x)')
    ax.plot(xxx, preds, label='~F(x)')
    ax.legend()
    return fig

In [ ]:
# model = BVPNet()
# model.init_weights('best_model.pth')
# model.eval()

# test_dataset = valid_dataset#train_dataset + valid_dataset
# test_loader = DataLoader(test_dataset)
# for i, (xxx, inputs, targets) in enumerate(tqdm(test_loader)):
#     xxx, inputs, targets = xxx[0], inputs[0], targets[0]
#     preds = model(inputs).detach()
#     fig = plot(xxx.numpy(), inputs.numpy(), targets.numpy(), preds.numpy())
#     fig.savefig(f'tests/{i}.png')
#     plt.close()